# 앙상블 검색
- 앙상블 검색은 희속 검색과 밀집 검색을 함께 쓰는 접근 방식입니다.
- 희소 검색의 빠르고 직관적인 검색, 밀집 검색의 관련성 판단 두 방식의 단점을 보완하고 균형잡힌 검색결과를 제공합니다.
- 희소 검색과 밀집 검색의 비중을 상황에 따라 조절할 수 있습니다.
- 기술 문서나 법률 문서와 같이 정확한 용어 매칭이 중요한 도메인에서는 희소 검색에 더 높은 가중치를 부여할 수 있습니다.
- 일반적인 질문-답변 시스템이나 의미 기반 추천 시스템에서는 밀집 검색에 더 높은 비중을 둘 수 있습니다.


In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = "data/투자설명서.pdf"
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = loader.load_and_split(doc_splitter)

- 희소 검색을 위한 BM25 리트리버를 정의합니다.

In [2]:
from langchain_community.retrievers import BM25Retriever
from kiwipiepy import Kiwi

kiwi_tokenizer= Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi_tokenizer.tokenize(text)]

In [3]:
bm25_retriever = BM25Retriever.from_documents(docs, preprocess_func=kiwi_tokenize)
bm25_retriever.k = 4

- 밀집 검색을 위한 FAISS DB와 리트리버를 구축합니다.

In [4]:
from langchain_ollama import OllamaEmbeddings
embedding = OllamaEmbeddings(model="bge-m3")

In [5]:
# FAISS 라이브러리 임포트
from langchain_community.vectorstores import FAISS

# FAISS DB 생성 후 저장
faiss_store = FAISS.from_documents(docs, embedding)
faiss_store.save_local("data/DB")

In [6]:
persist_directory = "data/DB"
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

In [7]:
# FAISS 리트리버 생성
faiss_retriever = vectordb.as_retriever(search_kwargs={"k": 4})

- 앞서 정의한 bm25 리트리버와 FAISS 리트리버를 묶어 앙상블 리트리버를 만듭니다.
- 비중은 1:1로 설정했으나 상황에 따라 조절할 수 있습니다.

In [8]:
from langchain.retrievers import EnsembleRetriever
ensemble_returever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5])

In [11]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

#관련 있는 문서 수집 후 최종 답변까지 수행
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini",temperature=0.2),
    chain_type="stuff",
    retriever=ensemble_returever,
    return_source_documents=True
)

In [12]:
qa_chain.invoke("이 회사가 발생한 주식의 총 발행량이 어느 정도야?")

{'query': '이 회사가 발생한 주식의 총 발행량이 어느 정도야?',
 'result': '회사가 현재까지 발행한 주식의 총수는 13,602,977주입니다.',
 'source_documents': [Document(metadata={'producer': 'iText® 5.5.9 ©2000-2015 iText Group NV (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2024-06-26T16:15:14+09:00', 'moddate': '2024-06-26T16:15:14+09:00', 'source': 'data/투자설명서.pdf', 'total_pages': 514, 'page': 29, 'page_label': '30'}, page_content='주4) "고위험고수익투자신탁등"이란 다음 각 호의 어느 하나에 해당하는 투자신탁 등을 말합니다.\n(1) 「조세특례제한법」 제91조의15 제1항에 따른 고위험고수익채권투자신탁(이하 "고위험고\n수익채권투자신탁"이라 함). 다만, 해당 투자신탁 등의 최초 설정일ㆍ설립일이 속하는 분기 또\n는 그 다음 분기 말일 전 배정하는 경우에는 같은 법 시행령 제93조 제1항 제1호 및 같은 조\n제5항에도 불구하고 배정일 직전 영업일의 고위험고수익채권의 보유비율이 같은 법 시행령 제\n93조 제1항 제1호 각 목의 비율 이상이어야 합니다.\n(2) 법률 제19328호「조세특례제한법」의 시행일 이전의 제91조의15제1항에 따른 고위험고\n수익투자신탁(이하 "고위험고수익투자신탁"이라 함)으로서 최초 설정일ㆍ설립일이 2023년\n12월 31일 이전인 것을 말합니다. 다만, 해당 투자신탁 등의 설정일ㆍ설립일부터 배정일까지\n의 기간이 6개월 미만일 경우에는 대통령령 제33499호 같은 법 시행령 시행일 이전의 제93조\n제3항 제1호 및 같은 조 제7항에도 불구하고 배정일 직전 영업일의 비우량채권과 코넥스 상장\n주식을 합한 보유비율이 100분의 45 이상이고 이를 포함한